In [ ]:
import scanpy as sc
import hacca
from hacca import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from STalign import STalign
## import dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import plotly
import requests
from STalign import STalign
# make plots bigger
import os, sys
print(sys.executable) # works this time
print(sys.version)
print(sys.version_info)
import scanpy as sc
from sklearn.cross_decomposition import CCA
from sklearn.manifold import TSNE
from scipy.sparse import csr_matrix
from scipy.sparse import csr_matrix
import numpy as np
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from STalign import STalign
## import dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import plotly
import requests

# make plots bigger
import os, sys
print(sys.executable) # works this time
print(sys.version)
print(sys.version_info)
import scanpy as sc
from sklearn.cross_decomposition import CCA
from sklearn.manifold import TSNE
from scipy.sparse import csr_matrix
from scipy.sparse import csr_matrix
import numpy as np
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler

In [7]:
def process(a,b_prime):
    import matplotlib.pyplot as plt
    xJ = a.D[:, 0]
    yJ = a.D[:, 1]
    xI = b_prime.D[:, 0]
    yI = b_prime.D[:, 1]
    XI,YI,I,fig = STalign.rasterize(xI,yI,dx=30,blur=1.5)
    XJ,YJ,J,fig = STalign.rasterize(xJ,yJ,dx=30, blur=1.5)
    extentI = STalign.extent_from_x((YI,XI))
    extentJ = STalign.extent_from_x((YJ,XJ))
    # run LDDMM
    # specify device (default device for STalign.LDDMM is cpu)
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    
    # keep all other parameters default
    params = {
                'niter': 1000,
                'device':device,
                'epV': 50
              }
    
    out = STalign.LDDMM([YI,XI],I,[YJ,XJ],J,**params)
    # get necessary output variables
    A = out['A']
    v = out['v']
    xv = out['xv']
    # set device for building tensors
    if torch.cuda.is_available():
        torch.set_default_device('cuda:0')
    else:
        torch.set_default_device('cpu')
        # apply transform
    phii = STalign.build_transform(xv,v,A,XJ=[YJ,XJ],direction='b')
    phiI = STalign.transform_image_source_to_target(xv,v,A,[YI,XI],I,[YJ,XJ])
    
    #switch tensor from cuda to cpu for plotting with numpy
    if phii.is_cuda:
        phii = phii.cpu()
    if phiI.is_cuda:
        phiI = phiI.cpu()
    # transform is invertible
    phi = STalign.build_transform(xv,v,A,XJ=[YI,XI],direction='f')
    phiiJ = STalign.transform_image_target_to_source(xv,v,A,[YJ,XJ],J,[YI,XI])
    
    #switch tensor from cuda to cpu for plotting with numpy
    if phi.is_cuda:
        phi = phi.cpu()
    if phiiJ.is_cuda:
        phiiJ = phiiJ.cpu()
    # apply transform to original points
    tpointsI= STalign.transform_points_source_to_target(xv,v,A, np.stack([yI, xI], 1).astype(np.double))
    
    #switch tensor from cuda to cpu for plotting with numpy
    if tpointsI.is_cuda:
        tpointsI = tpointsI.cpu()
    
    #switch from row column coordinates (y,x) to (x,y)
    xI_LDDMM = tpointsI[:,1]
    yI_LDDMM = tpointsI[:,0]
    df3 = pd.DataFrame(
    
        {
    
            "x": xI_LDDMM,
    
            "y": yI_LDDMM,
           
        },
    
    )
    b_prime.D=df3.to_numpy()
   

    return b_prime


STAlign

In [ ]:
samples = [(f"{prefix}1", f"{prefix}2") for prefix in "ABCDEFGHIJKLMN"]
base_path = "I:/mutiomics/Spatial Multimodal Analysis of Transcriptomes and Metabolomes in Tissues/benchmark/merge"
STalign_alignment_work_dir = os.path.join(base_path, 'STalign')
if not os.path.exists(STalign_alignment_work_dir):
    os.makedirs(STalign_alignment_work_dir)
accuracies = []
for sample in samples:
    print(f"process {sample}")
    sample1 = sample[0]
    sample2 = sample[1]
    a_h5ad = sc.read_h5ad(os.path.join(base_path, sample1+".h5ad"))
    b_prime_h5ad = sc.read_h5ad(os.path.join(base_path,sample2+".h5ad"))
    a_h5ad.obs["leiden"] = a_h5ad.obs["clusters"]
    b_prime_h5ad.obs["leiden"] = b_prime_h5ad.obs["clusters"]
    b_prime_spatial = pd.DataFrame(b_prime_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(b_prime_spatial)
    b_prime_spatial = pd.DataFrame(scaledata, columns=b_prime_spatial.columns).to_numpy()
    a_spatial = pd.DataFrame(a_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(a_spatial)
    a_spatial = pd.DataFrame(scaledata, columns=a_spatial.columns).to_numpy()
    a = Data(X=a_h5ad.X.toarray(), D = a_spatial, Label=a_h5ad.obs['leiden'].to_numpy())
    b_prime = Data(X=b_prime_h5ad.X.toarray(), D = b_prime_spatial, Label=b_prime_h5ad.obs['leiden'].to_numpy())
    b_truth = a # use a as the ground truth to evaluate b_predict
    _b_prime = process(a,b_prime)
    b_predict = hacca.direct_alignment(a, _b_prime, STalign_alignment_work_dir)
    pi = hacca.direct_alignment_metric(a, _b_prime)
    hacca.plot_b_predict(b_predict, STalign_alignment_work_dir)
    STalign_alignment_loss = hacca.loss(b_predict, b_truth)
    STalign_pairwise_alignment_accuracy = hacca.pairwise_alignment_accuracy(a, _b_prime, pi)
    print(f"{sample}, {STalign_alignment_loss},{STalign_pairwise_alignment_accuracy}")
    accuracies.append([sample, STalign_alignment_loss, STalign_pairwise_alignment_accuracy])

Directalign

In [ ]:
samples = [(f"{prefix}1", f"{prefix}2") for prefix in "ABCDEFGHIJKLMN"]
base_path = "I:/mutiomics/Spatial Multimodal Analysis of Transcriptomes and Metabolomes in Tissues/benchmark/merge"
direct_alignment_work_dir = os.path.join(base_path, 'STalign')
if not os.path.exists(direct_alignment_work_dir):
    os.makedirs(direct_alignment_work_dir)
Direct_accuracies = []
for sample in samples:
    print(f"process {sample}")
    sample1 = sample[0]
    sample2 = sample[1]
    a_h5ad = sc.read_h5ad(os.path.join(base_path, sample1+".h5ad"))
    b_prime_h5ad = sc.read_h5ad(os.path.join(base_path,sample2+".h5ad"))
    a_h5ad.obs["leiden"] = a_h5ad.obs["clusters"]
    b_prime_h5ad.obs["leiden"] = b_prime_h5ad.obs["clusters"]
    b_prime_spatial = pd.DataFrame(b_prime_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(b_prime_spatial)
    b_prime_spatial = pd.DataFrame(scaledata, columns=b_prime_spatial.columns).to_numpy()
    a_spatial = pd.DataFrame(a_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(a_spatial)
    a_spatial = pd.DataFrame(scaledata, columns=a_spatial.columns).to_numpy()
    a = Data(X=a_h5ad.X.toarray(), D = a_spatial, Label=a_h5ad.obs['leiden'].to_numpy())
    b_prime = Data(X=b_prime_h5ad.X.toarray(), D = b_prime_spatial, Label=b_prime_h5ad.obs['leiden'].to_numpy())
    b_truth = a # use a as the ground truth to evaluate b_predict
    b_predict = hacca.direct_alignment(a, b_prime, direct_alignment_work_dir)
    pi = hacca.direct_alignment_metric(a, b_prime)
    hacca.plot_b_predict(b_predict, direct_alignment_work_dir)
    direct_alignment_loss = hacca.loss(b_predict, b_truth)
    direct_alignment_pairwise_alignment_accuracy = hacca.pairwise_alignment_accuracy(a, b_prime, pi)
    hacca.plot_b_predict(b_predict, STalign_alignment_work_dir)
    print(f"{sample}, {direct_alignment_loss},{direct_alignment_pairwise_alignment_accuracy}")
    Direct_accuracies.append([sample, direct_alignment_loss, direct_alignment_pairwise_alignment_accuracy])

ICP align

In [ ]:
samples = [(f"{prefix}1", f"{prefix}2") for prefix in "ABCDEFGHIJKLMN"]
base_path = "I:/mutiomics/Spatial Multimodal Analysis of Transcriptomes and Metabolomes in Tissues/benchmark/merge"
icp_2d_work_dir = os.path.join(base_path, 'ICP2D')
if not os.path.exists(icp_2d_work_dir):
    os.makedirs(icp_2d_work_dir)
Direct_accuracies = []
for sample in samples:
    print(f"process {sample}")
    sample1 = sample[0]
    sample2 = sample[1]
    a_h5ad = sc.read_h5ad(os.path.join(base_path, sample1+".h5ad"))
    b_prime_h5ad = sc.read_h5ad(os.path.join(base_path,sample2+".h5ad"))
    a_h5ad.obs["leiden"] = a_h5ad.obs["clusters"]
    b_prime_h5ad.obs["leiden"] = b_prime_h5ad.obs["clusters"]
    b_prime_spatial = pd.DataFrame(b_prime_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(b_prime_spatial)
    b_prime_spatial = pd.DataFrame(scaledata, columns=b_prime_spatial.columns).to_numpy()
    a_spatial = pd.DataFrame(a_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(a_spatial)
    a_spatial = pd.DataFrame(scaledata, columns=a_spatial.columns).to_numpy()
    a = Data(X=a_h5ad.X.toarray(), D = a_spatial, Label=a_h5ad.obs['leiden'].to_numpy())
    b_prime = Data(X=b_prime_h5ad.X.toarray(), D = b_prime_spatial, Label=b_prime_h5ad.obs['leiden'].to_numpy())
    b_truth = a # use a as the ground truth to evaluate b_predict
    _b_prime = hacca.icp_2d_alignment(a, b_prime, icp_2d_work_dir)
    b_predict = hacca.direct_alignment(a, _b_prime, work_dir=icp_2d_work_dir)
    hacca.plot_b_predict(b_predict, icp_2d_work_dir)
    pi = hacca.direct_alignment_metric(a, _b_prime)
    icp_2d_loss = hacca.loss(b_predict, b_truth)
    ICP2D_pairwise_alignment_accuracy = hacca.pairwise_alignment_accuracy(a, _b_prime, pi)
    hacca.plot_b_predict(b_predict, STalign_alignment_work_dir)
    print(f"{sample}, {icp_2d_loss},{ICP2D_pairwise_alignment_accuracy}")
    Direct_accuracies.append([sample, icp_2d_loss, ICP2D_pairwise_alignment_accuracy])


haCCA

In [ ]:
samples = [(f"{prefix}1", f"{prefix}2") for prefix in "M"]
base_path = "I:/mutiomics/Spatial Multimodal Analysis of Transcriptomes and Metabolomes in Tissues/benchmark/merge"
haCCA_work_dir = os.path.join(base_path, 'haCCA')
if not os.path.exists(haCCA_work_dir):
    os.makedirs(haCCA_work_dir)
Direct_accuracies = []
for sample in samples:
    print(f"process {sample}")
    sample1 = sample[0]
    sample2 = sample[1]
    a_h5ad = sc.read_h5ad(os.path.join(base_path, sample1+".h5ad"))
    b_prime_h5ad = sc.read_h5ad(os.path.join(base_path,sample2+".h5ad"))
    a_h5ad.obs["leiden"] = a_h5ad.obs["clusters"]
    b_prime_h5ad.obs["leiden"] = b_prime_h5ad.obs["clusters"]
    b_prime_spatial = pd.DataFrame(b_prime_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(b_prime_spatial)
    b_prime_spatial = pd.DataFrame(scaledata, columns=b_prime_spatial.columns).to_numpy()
    a_spatial = pd.DataFrame(a_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(a_spatial)
    a_spatial = pd.DataFrame(scaledata, columns=a_spatial.columns).to_numpy()
    a = Data(X=a_h5ad.X.toarray(), D = a_spatial, Label=a_h5ad.obs['leiden'].to_numpy())
    b_prime = Data(X=b_prime_h5ad.X.toarray(), D = b_prime_spatial, Label=b_prime_h5ad.obs['leiden'].to_numpy())
    b_truth = a # use a as the ground truth to evaluate b_predict

    _b_prime = hacca.manual_gross_alignment(a, b_prime, work_dir=haCCA_work_dir)
    _b_prime = hacca.further_alignment(a, _b_prime)
    _a, _b_prime = hacca.icp_2d_with_feature_alignment(a, _b_prime, dist_min=1, simpson_index_threshold=0.8)
    b_preidct = hacca.direct_alignment(_a, _b_prime, work_dir=haCCA_work_dir)
    b_predict_metric = hacca.direct_alignment_metric(_a, _b_prime)
    haCCA = hacca.loss(b_preidct, b_truth)
    hacca.plot_b_predict(b_preidct,haCCA_work_dir)
    hacca_pairwise_alignment_accuracy = hacca.pairwise_alignment_accuracy(_a, _b_prime, b_predict_metric)
    print(f"{sample}, {haCCA},{hacca_pairwise_alignment_accuracy}")
    Direct_accuracies.append([sample, haCCA, hacca_pairwise_alignment_accuracy])


Directalign

In [ ]:
samples = [(f"{prefix}1", f"{prefix}2") for prefix in "LMN"]
base_path = "I:/mutiomics/Spatial Multimodal Analysis of Transcriptomes and Metabolomes in Tissues/benchmark/merge"
manual_alignment_work_dir = os.path.join(base_path, 'manual_alignment')
if not os.path.exists(manual_alignment_work_dir):
    os.makedirs(manual_alignment_work_dir)
Direct_accuracies = []
for sample in samples:
    print(f"process {sample}")
    sample1 = sample[0]
    sample2 = sample[1]
    a_h5ad = sc.read_h5ad(os.path.join(base_path, sample1+".h5ad"))
    b_prime_h5ad = sc.read_h5ad(os.path.join(base_path,sample2+".h5ad"))
    a_h5ad.obs["leiden"] = a_h5ad.obs["clusters"]
    b_prime_h5ad.obs["leiden"] = b_prime_h5ad.obs["clusters"]
    b_prime_spatial = pd.DataFrame(b_prime_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(b_prime_spatial)
    b_prime_spatial = pd.DataFrame(scaledata, columns=b_prime_spatial.columns).to_numpy()
    a_spatial = pd.DataFrame(a_h5ad.obsm['spatial'])
    scaledata = hacca.center_and_scale(a_spatial)
    a_spatial = pd.DataFrame(scaledata, columns=a_spatial.columns).to_numpy()
    a = Data(X=a_h5ad.X.toarray(), D = a_spatial, Label=a_h5ad.obs['leiden'].to_numpy())
    b_prime = Data(X=b_prime_h5ad.X.toarray(), D = b_prime_spatial, Label=b_prime_h5ad.obs['leiden'].to_numpy())
    b_truth = a # use a as the ground truth to evaluate b_predict

    _b_prime = hacca.manual_gross_alignment(a, b_prime, work_dir=manual_alignment_work_dir)
    _b_prime = hacca.further_alignment(a, _b_prime)
    manual_alignment = hacca.loss(b_preidct, b_truth)
    manual_alignment_pairwise_alignment_accuracy = hacca.pairwise_alignment_accuracy(a, _b_prime, pi)
    print(f"{sample}, {manual_alignment},{manual_alignment_pairwise_alignment_accuracy}")
    Direct_accuracies.append([sample, manual_alignment, manual_alignment_pairwise_alignment_accuracy])

In [ ]:
accuracies_df = pd.DataFrame(Direct_accuracies, columns=['Sample', 'Alignment_Loss', 'Pairwise_Alignment_Accuracy'])
accuracies_df[['Alignment_Loss_1', 'Alignment_Loss_2', 'Alignment_Loss_3']] = pd.DataFrame(accuracies_df['Alignment_Loss'].tolist(), index=accuracies_df.index)
accuracies_df.drop(columns=['Alignment_Loss'], inplace=True)
accuracies_df

In [ ]:
pi = hacca.direct_alignment_metric(a, _b_prime)
hacca.plot_b_predict(b_predict, STalign_alignment_work_dir)
STalign_alignment_loss = hacca.loss(b_predict, b_truth)
STalign_pairwise_alignment_accuracy = hacca.pairwise_alignment_accuracy(a, _b_prime, pi)
print(f"Run STalign_alignment: loss: {STalign_alignment_loss}, pairwise_alignment_accuracy: {STalign_pairwise_alignment_accuracy}")